# Data exploration and analysis: GooglePlayStoreApps

With a dataset with information from the applications in the Google Play Store, the aim is to obtain a model capable of predicting the rating for applications not yet rated by users.
Since the rating can be a real number between 1 and 5, this will be considered a regression problem.

There are two datasets:
- one with the data of the apps in the playstore, including information about the applications (such as category, rating and number of reviews, among others);
- another with the reviews made by users, including the text and the value of the sentiment/polarity identified in those comments.

To create the model, a series of tasks will first be carried out:
- explore the data and relationships between different features, identifying missing values and outliers;
- creation of a pipeline with the tasks to perform prior to model training;
- creation of various models, training and testing;
- use of metrics to select the final model.

# Tabla de contenidos

1. [Exploratory data analysis](#eda)
    1. [Application data](#dataapps)
    2. [Eliminate duplicate rows](#removeduplicates)
    3. [Dealing with missing values](#missingvalues)
    4. [Analysis of numerical features](#numericfeatures)
    5. [Analysis of categorical features](#categoricalfeatures) 

In [ ]:
# Imports

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Reading the files.

# -- Data about apps.
data_apps = pd.read_csv("data/googleplaystore.csv")

# -- Reviews made by users and sentiment analysis values.
data_reviews = pd.read_csv("data/googleplaystore_user_reviews.csv")

#### Regarding the reviews dataset

We are going to concentrate mainly on the data of the apps (data_apps) to try to predict the rating. While the data about reviews (the polarity of mixed sentiments in those reviews) might help us predict a Rating, for now we're going to use only app values to do so.

Especially since we only have data from 1074 apps, compared to the 8182 apps in the final dataset (removed duplicate rows and also apps without Rating) => reviews for less than 15% of the apps.

In [ ]:
## Information about de reviews
data_reviews

In [ ]:
data_reviews.groupby("App").count()

# Exploratory data analysis <a name="eda"></a>

Before creating the model, we will analyze the data. This involves looking at the values they can take, recategorizing the data if necessary, and looking at missing values and outliers and dealing with them in some way.

## Application data <a name="dataapps"></a>

In this dataset we have the following features:
- App: indicates the name of the application.
- Category: the category of the app (Art and design, Family, Sports, etc.).
- Rating (variable to predict): indicates the rating of the application, a real value (with a decimal) that can range from 1 to 5.
- Reviews: the number of reviews that the app has.
- Size: the size of the application.
- Installs: the number of installations of the apps.
- Type: if it is free or paid.
- Price: the price.
- Content Rating: the content rating (suitable for everyone, over 18, etc).
- Genres: the genres of the application.
- Last Updated: when it was last updated.
- Current Ver: the version of the application.
- Android Ver: the version of android in which it can be used.

In [ ]:
data_apps

Let's see the information about the type of the features.
We have 10842 rows. Most features are type Object. Rating is a float, and Reviews is an integer. Other features that could be numeric are:
- Size
- Installs
- Price

In [ ]:
data_apps.info()

We can also see that we have some missing values in some columns. We will deal with these missing values later.

## Removing duplicate rows <a name="removeduplicates"></a>

Before we start dealing with missing values and looking at features, let's make sure we don't have duplicate rows for apps.

Number of duplicate rows: 1181

In [ ]:
# Duplicated rows for app name => 1181
data_apps[data_apps.duplicated(['App'])]

Example with ROBLOX.
We can see that the number of reviews is the value that is changing => regarding that, we can keep the highest.
It also changes the category (!). Let's see if this happens for many apps.

In [ ]:
data_apps[data_apps["App"] == "ROBLOX"].sort_values(by="Reviews", ascending=False)

How many apps have multiple categories? => 85.

Since there are just a few, in this case we are going to choose to keep the last category that was assigned to the app.
We are left with the most up-to-date entry regarding "Reviews".

In [ ]:
temp = data_apps.groupby(["App"])["Category"].unique()
temp[temp.str.len() > 1]

In [ ]:
data_apps = data_apps.sort_values('Reviews', ascending=False).drop_duplicates('App').sort_index()

# Example
data_apps[data_apps["App"] == "ROBLOX"]

### Dealing with missing values <a name="missingvalues"></a>
 
We are now going to deal with the missing values in the features. We first calculate the sum of the missing values in each feature, ordered in descending order.

In [ ]:
data_apps.isna().sum().sort_values(ascending=False)

We have quite a few missing values in Rating, which is the variable to predict, and a few others in Current Ver, Android Ver, Category, Type and Genres. Let's observe and try to solve these missing values.

Let's start with those that only have one missing value: Category, Type, and Genres.

In [ ]:
# Let's see the row with missing value in Type
data_apps[data_apps["Type"].isna()]

In [ ]:
# What kind of values can Type have? ==> "Free" or "Paid".
data_apps["Type"].unique()

Since Type can only take the values "Free" or "Paid", and the price of that app is 0, we can fill in the missing value as "Free".

In [ ]:
# Updating the value of the row 9148, column 6 (Type) por Free
data_apps.iloc[[9148],[6]] = "Free"

# data_apps[data_apps["Type"].isna()] # To check that we don't have that values

Now let's look at the row that has a missing value in Category.
Looking at the row, we can also see that it has a missing value for Genres.

A possible solution could be to delete this row. But first let's see what kind of categories and genres exist, to see if there is any category "Others" or something like that.

In [ ]:
data_apps[data_apps["Category"].isna()]

In [ ]:
# Possible values for Category

data_apps["Category"].unique()

In [ ]:
data_apps["Category"].value_counts()

Let's look at the possible values for Genres. These seem to be overlapping quite a bit with the Category. You can see that all the values for Category appear as Genres. There are also new values for Genres that seem to indicate subcategories (for example, different types of games).

In [ ]:
data_apps["Genres"].unique()

There doesn't seem to be a category like "Other". Let's delete this row.

In [ ]:
# Removing the row with missing value in Category and Genres
data_apps = data_apps.drop([10472])

Let's see the rows with missing values in Android View, and the possible values that feature can take.

In [ ]:
data_apps[data_apps["Android Ver"].isna()]

In [ ]:
data_apps["Android Ver"].unique()

For simplicity, let's fill in the missing values with the value "Varies with device".

In [ ]:
data_apps["Android Ver"] = data_apps["Android Ver"].fillna(value="Varies with device")

Let's look at the rows with missing values in Current Ver, and also the possible values for Current Ver. There is also a "Varies with device" value here. For simplicity, we also fill in these missing values with "Varies with device".

In [ ]:
data_apps[data_apps["Current Ver"].isna()]

In [ ]:
data_apps["Current Ver"].value_counts()

In [ ]:
data_apps["Current Ver"] = data_apps["Current Ver"].fillna(value="Varies with device")

We still need to see what happens to the missing values for our variable to predict, "Rating". Let's see how many Reviews are associated with it.
- Rows that lack rating: 1474
- Rows that lack rating and have some review: 878
- Rows that lack rating and have no reviews: 596

Not all of them have reviews to be able to use those values of the reviews (from the other dataset) in order to predict some Rating.

In [ ]:
# rows a las que les falta rating: 1474 
# data_apps[data_apps["Rating"].isna()] 

# rows a las que les falta rating y tienen algún review: 878 
# data_apps[(data_apps["Rating"].isna()) & (data_apps["Reviews"] > 0)] 

# rows a las que les falta rating y no tienen reviews: 596  
# data_apps[(data_apps["Rating"].isna()) & (data_apps["Reviews"] == 0)] 

In this case we have two options:
- delete rows;
- use some Imputer to complete the data (KNNImputer, for example) => taking into account what this means (completing the missing data using the nearest neighbors).

Let's try both approaches and compare the results.

## Analysis of numeric features <a name="numericfeatures"></a>

Now that we've dealt with missing values, let's move on to the analysis of numerical features:
- Rating
- Reviews
- Size
- Installs
- Price

This analysis also implies recategorization in the dataset, since many of these were type Object and must be converted to numerical.

Let's start by looking at how the Rating values behave. We see that Rating has average values of 4.4, and most of the apps have a rating greater than 3.3. We have few examples for apps with poor rating.

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(20,6));
sns.histplot(data=data_apps, x="Rating", kde=True, ax=axes[0])
sns.boxplot(data=data_apps, x="Rating", ax=axes[1])

In [ ]:
# data_apps["Rating"].median()
data_apps["Rating"].describe()

Let's see if there is a relationship between the number of reviews and the rating. The graph seems to indicate that apps with a rating between 4 and 4.7 have a much higher number of reviews; kind of follows the same distribution as Rating.

In [ ]:
plt.figure(figsize=(20,6), dpi=150);
sns.barplot(data=data_apps, x="Rating", y="Reviews")

Let's take a look at the reviews. In the first plot we can see the boxplot for all the values, and we can observe some outliers, some apps with a very high number of reviews. Most have much less.

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(20,6), dpi=150)
sns.boxplot(data=data_apps, x="Reviews", ax=axes[0])
sns.boxplot(data=data_apps, x="Reviews", ax=axes[1])
plt.xlim(-5000, 500000)

Here we can see those apps with many reviews, such as Facebook, Whatasapp, Instagram. In this case, our outliers in Reviews are well-known and popularly used Apps. However, they represent phenomena that can happen with some apps, so we're not going to remove them from the dataset.

Perhaps we can try retraining the models without information about these apps, and evaluate the differences.

In [ ]:
data_apps.sort_values(by="Reviews", ascending=False).head(5)

For the feature Size we have a series of values in MB (M) and kB (k), and many entries with "Varies with device".
To convert it into a numeric variable, we are going to keep only the MB, doing a conversion for those in kB.
The rows with value "Varies with device" will be completed with the mean size.

In [ ]:
data_apps["Size"].value_counts()
#data_apps["Size"].unique()

In [ ]:
# Function to obtain a numerical value for the feature Size.
# If it has a "M", we keep it that way.
# If it has a "k", we convert it to MB.
# If the value is "Varies with device", we return np.NaN to later fill it with the mean size.
def from_kb_to_mb(size):
    if "M" in size:
        return float(size[:-1]) # we skip last char => M
    if "k" in size:
        return round(float(size[:-1]) / 1024) # round => to have only 1 decimal, just as M has
    
    return np.NaN

In [ ]:
data_apps["Size"] = data_apps["Size"].map(from_kb_to_mb)

mean_size = round(data_apps["Size"].mean(), 1)
data_apps["Size"] = data_apps["Size"].fillna(value=mean_size)

Now that the feature is numeric, let's look at a boxplot. Most apps are less than 50MB in size.

In [ ]:
sns.boxplot(data=data_apps, x="Size")

Let's see if there is a relationship between Size and Rating. No trend or clear relationship between the two is observed.

In [ ]:
plt.figure(figsize=(20,6), dpi=150);
sns.barplot(data=data_apps, x="Rating", y="Size")

Let's now analyze the "Installs" feature. It has a series of categorical values (more than 10000 installs, etc). Since there is an order between them, we are going to change it to numerical values, removing the "+" from all of them.

In [ ]:
data_apps["Installs"].value_counts()

In [ ]:
# Function to return the value of "installs" as numerical value
def installs_to_num(installs):
    if "+" in installs:
        installs = installs[:-1]
    installs = installs.replace(",", "")
    return int(installs)

data_apps["Installs"] = data_apps["Installs"].map(installs_to_num)

Let's take a look at how Installs behaves. Here we also have some outliers with a large number of installs. Possibly the same thing that happens with reviews for very popular apps like Facebook or Instagram.

In [ ]:
plt.figure(figsize=(20,6), dpi=150);
sns.boxplot(data=data_apps, x="Installs")

In [ ]:
# data_apps.sort_values(by="Installs", ascending=False).head(5)
# data_apps["Installs"].value_counts()

sns.countplot(data=data_apps, x="Installs")
plt.xticks(rotation=90);

Let's observe the relationship between Installs and Rating ==> there is a growing trend, the more installations, the more rating (with some exceptions with few installations).

In [ ]:
plt.figure(figsize=(20,6), dpi=150);
sns.barplot(data=data_apps, x="Installs", y="Rating")
plt.xticks(rotation=45);

Now let's see what happens with the "Price" feature. Here the "$" symbol is used, which makes it impossible for us to treat the feature as numeric, so we are going to remove it.

In [ ]:
data_apps["Price"].value_counts()

In [ ]:
# Function to remove the "$" symbol.
def price_to_num(price):
    return float(price.replace("$", ""))

data_apps["Price"] = data_apps["Price"].map(price_to_num)

In [ ]:
# Let's see the boxplot of the price (greater that 0)
fig, axes = plt.subplots(1,2, figsize=(20,6));
sns.boxplot(data=data_apps[data_apps["Price"] > 0], x="Price", ax=axes[0])
sns.histplot(data=data_apps, x=data_apps["Price"], ax=axes[1])

We have some very expensive apps. Looking at the dataset sorted by Price (highest to lowest) we see that those apps over $300 appear to be "meme" apps. At least they all seem to have the same spirit.

We are going to remove these outliers (those over 299.99).

In [ ]:
data_apps.sort_values(by="Price", ascending=False).head(20)

In [ ]:
# Removing the outliers => those apps with price greater than $200, that are all "I am rich" apps variations
data_apps = data_apps.drop(data_apps[data_apps["Price"] > 200].index)

In [ ]:
# Boxplot without outliers
# Most of the apps cost less than $10
plt.figure(figsize=(20,6));
sns.boxplot(data=data_apps[data_apps["Price"] > 0], x="Price")

In [ ]:
# Relation between Price and Rating. We can't observe a clear relationship.
plt.figure(figsize=(20,6), dpi=150);
sns.barplot(data=data_apps, x="Rating", y="Price")
plt.xticks(rotation=90);

Although Last Updated is not numeric, it is ordinal => let's pass it to datetime and see if there is any trend.

In [ ]:
data_apps['Last Updated'] = pd.to_datetime(data_apps['Last Updated'], format="%d/%m/%Y")
data_apps["Last Updated"] 

In [ ]:
plt.figure(figsize=(20,6), dpi=200);
sns.histplot(data=data_apps, x="Last Updated", kde=True)

Let's simplify to checking only the years. We can see that most of the apps were recently updated.

In [ ]:
data_apps["Year Updated"] = data_apps["Last Updated"].dt.year
sns.countplot(data=data_apps, x="Year Updated")

In [ ]:
# Relation between year of last update and rating of the app.
fig, axes = plt.subplots(1, 2, figsize=(20, 6));
sns.boxplot(data=data_apps, x="Year Updated", y="Rating", ax=axes[0])
sns.violinplot(data=data_apps, x='Year Updated', y='Rating', ax=axes[1])

For Current Ver we can see that we have a wide variety of possible versions. Let's simplify this data by using only the first number as the version to get an idea of how the data is distributed.

In [ ]:
data_apps["Current Ver"].value_counts()
#sns.boxplot(x=data_apps["Current Ver"])

In [ ]:
import re 

def simplified_app_version(version):
    if type(version) != str:
        return version
    
    results = re.findall(r'(\d)[\.\d+]*', version)
    if results:
        return int(results[0])
    else: # All the "Varies with device" and the outliers (as "final", "DH-Security Camera", etc)
        return np.NaN

data_apps["Simplified App Ver"] = data_apps["Current Ver"].map(simplified_app_version)
data_apps["Simplified App Ver"].value_counts()

In [ ]:
# Most of the versions are between 0 and 6. Higher versions are outliers.
sns.boxplot(data=data_apps, x="Simplified App Ver")

In [ ]:
# We completed some versions as "na", let's count how many.
data_apps["Simplified App Ver"].isna().sum()

In [ ]:
# Let's see if the version of the app impacts in the rating. We can't observe any relation between both.
sns.regplot(data=data_apps, x="Simplified App Ver", y="Rating") 

In [ ]:
# Relation between version and reviews
sns.regplot(data=data_apps, x="Simplified App Ver", y="Reviews")

We are not going to use the version of the app to predict the Rating. Let's see what happens with the Android version.

Here we also have to clean up the data a bit. We are going to simplify the version by using only the first number and assuming that the version implies that it accepts that number and those that follow.

In [ ]:
data_apps["Android Ver"].value_counts()

In [ ]:
# We can simplify the version using only the first number
def simplified_android_version(version):
    if type(version) != str:
        return version
    results = re.findall(r'(\d+)[\.\d+]*', version)
    if results:
        return int(results[0])
    else: # all those that have "Varies with device"
        return np.NaN

data_apps["Simplified Android Ver"] = data_apps["Android Ver"].map(simplified_android_version)

In [ ]:
# Most of the apps have android version 4 or higher
data_apps["Simplified Android Ver"].value_counts()

In [ ]:
sns.histplot(data=data_apps, x="Simplified Android Ver", cumulative=True) 
# cumulative=True given that Simplified Android Ver means that works for that version and higher

Let's see if there is a relationship with the Rating. We can see that as the version is higher/newer, the higher the user scores tend to be.

In [ ]:
sns.boxenplot(data=data_apps, x="Simplified Android Ver", y="Rating") 

In [ ]:
# data_apps[data_apps["Simplified Android Ver"].isna()]

We have some missing values. Let's complete the Android version with the average Android version per year.

In [ ]:
group_year_agg_android = data_apps.groupby(["Year Updated"]).agg({"Simplified Android Ver": ["mean"]})
group_year_agg_android

In [ ]:
def replace_varies_android_ver(row):
    if pd.isna(row["Simplified Android Ver"]):
        year = row["Year Updated"]
        return round(group_year_agg_android.loc[year, ("Simplified Android Ver", "mean")])
    else:
        return row["Simplified Android Ver"]
    
data_apps["Simplified Android Ver"] = data_apps.apply(replace_varies_android_ver, axis=1)

Let's see if there are correlations between the numerical variables.

In [ ]:
data_apps.corr()

In [ ]:
# The strongest correlations are between the number of Installs and Reviews; between Year Updated and Simplified Android View 
# --which makes sense; and then weaker correlations between Year Updated and Size, and Year Updated and Rating, Size and 
#Simplified Android Ver.

# Price is the one that seems least correlated with the rest.
sns.heatmap(data_apps.corr(), cmap="Paired", annot=True)

## Analysis of categorical features <a name="categoricalfeatures"></a>

Let's see how categorical features behave and what we should take into account. These features are:
- Category
- Genres
- Type
- Content Rating

Let's start with the Category and Genres features. Most popular categories: FAMILY, GAME and TOOLS.

In [ ]:
# data_apps["Category"].value_counts()
plt.figure(figsize=(20,6), dpi=150)
sns.countplot(data=data_apps, x="Category")
plt.xticks(rotation=90);

Let's look at the relationship between Category and Rating, and then look at the top rated categories on average: EVENTS, EDUCATION, ART_AND_DESIGN.

In [ ]:
plt.figure(figsize=(20,6), dpi=200)
sns.boxplot(x="Category", y="Rating", data=data_apps)
plt.xticks(rotation=90);

In [ ]:
# Let's sort by Category best scored on average.
data_apps.groupby(["Category"]).agg({"Rating": {"mean", "min", "max"}}).sort_values(by=("Rating", "mean"), ascending=False)

Let's observe Relationship between Category and number of Reviews. Those categories with the highest number of reviews are SOCIAL, COMMUNICATION and GAME.

In [ ]:
plt.figure(figsize=(20,6), dpi=200)
sns.barplot(x="Category", y="Reviews", data=data_apps)
plt.xticks(rotation=90);

Let's take a look at what happens with the genres of an application. Looking at the possible values of the genres, it seems that they generally share the same value as the category or specify some sub-genre within that category (such as "Puzzle" to define some type of game).

In "Genres" the genres are separated by ";", and some apps have two genres. We are going to save in other columns the "Main Genre" (the first) and the "Secondary Genre" (the second).

In [ ]:
def get_main_genre(genres):
    genres = genres.split(";")
    return genres[0]
    
def get_secondary_genre(genres):
    genres = genres.split(";")
    second_g = ""
    if len(genres) > 1:
        return genres[1]
        
    return ""

data_apps["Main Genre"] = data_apps["Genres"].apply(get_main_genre)
data_apps["Secondary Genre"] = data_apps["Genres"].apply(get_secondary_genre)

We are going to evaluate the number of apps whose Category and Main Genre is the same. More than half of the time, the category coincides with the first assigned gender, so there is duplication of information. But let's see what happens to the ones that don't match.

In [ ]:
def the_same(dataframe):
    category = dataframe["Category"].replace("_", " ").replace("AND",  "&").lower()
    main_genre = dataframe["Main Genre"].lower()
    
    return category == main_genre

data_apps["the_same"] = data_apps.apply(the_same, axis=1)
data_apps["the_same"].value_counts()

For apps whose Category and Main Genre do not match:

- In VIDEO_PLAYERS, the genre is Video Players & Editors.
- In GAME, genres are game subcategories/genres.
- In FAMILY, the genres are very varied.

In [ ]:
group = data_apps[~data_apps["the_same"]].groupby(["Category", "Main Genre"])
group.size()

Let's see if the Main Genre can tell us anything about the ratings.

Although "Events" is the best punctuated on average (and this was already told us by Category), new information now appears, such as Word and Puzzle very well punctuated (within the "Game" Category).

In [ ]:
plt.figure(figsize=(20,6), dpi=200)
sns.boxplot(x="Main Genre", y="Rating", data=data_apps)
plt.xticks(rotation=90);

Let's see the relationship between the Main Genre and the number of Reviews. We can observe subgenres with many reviews such as Strategy, Arcade, Casual, among others.

In [ ]:
plt.figure(figsize=(20,6), dpi=200)
sns.barplot(x="Main Genre", y="Reviews", data=data_apps)
plt.xticks(rotation=90);

Regarding the Secondary Genre, since only a very small percentage of the data has a secondary genre, we will not use this category.

In [ ]:
number_rows = 9643
print(f"Percentage of apps without Secondary Genre: {round((len(data_apps[data_apps['Secondary Genre'] == '']) / number_rows) * 100, 2)}%") 
print(f"Percentage of apps with Secondary Genre: {round((len(data_apps[data_apps['Secondary Genre'] != '']) / number_rows) * 100, 2)}%") 

#data_apps.groupby(["Secondary Genre"]).agg({"Rating": {"mean", "min", "max", "std"}}).sort_values(by=("Rating", "mean"), ascending=False)

Now let's see the "Type" feature. Most of the apps are free. Paid apps tend to have a slightly better rating than free ones.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 6));
sns.countplot(data=data_apps, x="Type", ax=axes[0])
sns.boxplot(data=data_apps, x="Type", y="Rating", ax=axes[1])

Let's now see "Content Rating", and its relationship with "Rating". Depending on the Content Rating of the operation, some final ratings are better (such as Adults only 18+).

In [ ]:
# data_apps["Content Rating"].value_counts()
sns.countplot(x=data_apps["Content Rating"])
plt.xticks(rotation=90);

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 6));
sns.boxplot(data=data_apps, x="Content Rating", y="Rating", hue="Type", ax=axes[0])
sns.violinplot(data=data_apps, x='Content Rating', y='Rating', hue="Type", ax=axes[1])

In [ ]:
data_apps[data_apps["Content Rating"] == "Unrated"]
# data_apps[data_apps["Content Rating"] == "Adults only 18+"]

Let's change the outliers from "Unrated" to "Everyone" for these two cases.

In [ ]:
data_apps.loc[data_apps["Content Rating"] == "Unrated", "Content Rating"] = "Everyone"